&emsp;搭建一个简单的三层网络，包括一个输入层，一个隐含层，一个输出层

In [1]:
import tensorflow as tf
import numpy as np

&emsp;创建一个函数add_layer用于添加神经网络层：  
&emsp;输入有&emsp;1、inputs:代表该层的输入数据&emsp;2、in_size:输入的属性个数，也就是该层神经元数&emsp;3、out_size:输出个数，对应下层的in_size&emsp;4、activation:激励函数，用来实现非线性   

In [2]:
def add_layer(inputs, in_size, out_size, activation = None):
    Weight = tf.Variable(tf.random_normal([in_size, out_size]))
    Bias = tf.Variable(tf.zeros([1, out_size]) + 0.1) 
    Y = tf.matmul(inputs, Weight) + Bias
    if activation == None:
        outputs = Y
    else:
        outputs = activation(Y)
    return outputs

&emsp;生成训练数据x_data与之对应的y_data，并加入了一些随机值噪音。x_data共300个样本，值位于-1与1之间  
&emsp;这里用到了tf.placeholder，这是tensorflow的占位符，通常用于为输入数据占位  
&emsp;[:,np.newaxis]是为x_data的数据增加一个维数，这里其实起转置的作用

In [3]:
xs = tf.placeholder(tf.float32, [None, 1])
ys = tf.placeholder(tf.float32, [None, 1])
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
#效果等同于：
#x_data = np.linspace(-1, 1, 300)
#x_data = np.transpose([x_data])
#注意np.transpose只对矩阵有用，而linspace生成的是一个向量，shape为(300，)，需要加一个[]将其shape变为(1,300)
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

&emsp;$y=Wx+b$  式中特征属性只有一个$W$，因此输入层神经元只用一个  
&emsp;隐含层的神经元个数不定，如何设定可以获得最好效果至今尚未有精确的方法，这里设为10个

In [4]:
hiden = add_layer(xs, 1, 10, tf.nn.relu)
out = add_layer(hiden, 10, 1, activation = None)

&emsp;定义损失函数与优化方法，reduce_sum第二个参数表示在某一维求和，若无则对所有的值求和，这个参数对所有reduce_xxx方法具有同样意义

In [5]:
loss = tf.reduce_mean((tf.reduce_sum(tf.square(ys - out), 1)))
train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
init = tf.global_variables_initializer()

&emsp;利用会话启动网络，每次调用含有placeholder的方法时都必须用feed_dict为placeholder赋值

In [6]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1000):
        sess.run(train, feed_dict = {xs : x_data, ys : y_data})
        if step % 50 == 0:
            print(sess.run(loss, feed_dict = {xs : x_data, ys : y_data}))

0.245179
0.0106736
0.00646235
0.00566805
0.00534444
0.00513883
0.00496689
0.00481306
0.0046786
0.00455677
0.00444206
0.00433182
0.00422759
0.00412685
0.00402952
0.00393568
0.00384559
0.00375995
0.00367748
0.00360045
